In [5]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
!pip install nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')


[nltk_data] Downloading package stopwords to C:\Users\Akarsh
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to C:\Users\Akarsh
[nltk_data]     Kumar\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to C:\Users\Akarsh
[nltk_data]     Kumar\AppData\Roaming\nltk_data...


True

In [6]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
252,It was a Sunday night and I was waiting for th...,negative
962,Now I had the pleasure of first viewing Contam...,positive
902,The film disappointed me for many reasons: fir...,negative
877,This movie is George C. Scott at his very best...,positive
781,I have this movie on a collection of inexpensi...,positive


In [7]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

<>:32: SyntaxWarning: invalid escape sequence '\s'
<>:32: SyntaxWarning: invalid escape sequence '\s'
C:\Users\Akarsh Kumar\AppData\Local\Temp\ipykernel_21384\3798096103.py:32: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()


In [8]:
df = normalize_text(df)
df.head()

,review,sentiment
252,sunday night waiting advertised movie tv said ...,negative
962,pleasure first viewing contaminated man premie...,positive
902,film disappointed many reason first depiction ...,negative
877,movie george c scott best bernard hughes best ...,positive
781,movie collection inexpensive b movie restored ...,positive


In [9]:
df['sentiment'].value_counts()

sentiment
positive    256
negative    244
Name: count, dtype: int64

In [10]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [11]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
252,sunday night waiting advertised movie tv said ...,0
962,pleasure first viewing contaminated man premie...,1
902,film disappointed many reason first depiction ...,0
877,movie george c scott best bernard hughes best ...,1
781,movie collection inexpensive b movie restored ...,1


In [12]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [13]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [22]:
import dagshub
%pip install -q dagshub mlflow

mlflow.set_tracking_uri('https://dagshub.com/Akarsh-Kumar-2004/reimagined-waffle.mlflow')
dagshub.init(repo_owner='akarsh-kumar-2004', repo_name='reimagined-waffle', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Note: you may need to restart the kernel to use updated packages.


Initialized MLflow to track repo "akarsh-kumar-2004/reimagined-waffle"

Repository akarsh-kumar-2004/reimagined-waffle initialized!

2026/01/10 00:59:56 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/e742a002c2cc4624a4c7d62940a5525b', creation_time=1767986997098, experiment_id='0', last_update_time=1767986997098, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [23]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2026-01-10 01:00:13,622 - INFO - Starting MLflow run...
2026-01-10 01:00:14,130 - INFO - Logging preprocessing parameters...
2026-01-10 01:00:15,180 - INFO - Initializing Logistic Regression model...
2026-01-10 01:00:15,180 - INFO - Fitting the model...
2026-01-10 01:00:15,212 - INFO - Model training complete.
2026-01-10 01:00:15,214 - INFO - Logging model parameters...
2026-01-10 01:00:15,538 - INFO - Making predictions...
2026-01-10 01:00:15,540 - INFO - Calculating evaluation metrics...
2026-01-10 01:00:15,557 - INFO - Logging evaluation metrics...
2026-01-10 01:00:17,034 - INFO - Saving and logging the model...
2026/01/10 01:00:17 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2026-01-10 01:00:31,302 - INFO - Model training and logging completed in 17.17 seconds.
2026-01-10 01:00:31,304 - INFO - Accuracy: 0.624
2026-01-10 01:00:31,306 - INFO - Precision: 0.6964285714285714
2026-01-10 01:00:31,309 - INFO - Recall: 0.5652173913043478
2026-01-10

🏃 View run dashing-worm-942 at: https://dagshub.com/Akarsh-Kumar-2004/reimagined-waffle.mlflow/#/experiments/0/runs/e03677ca2c5f4c9db0b73287daaedce9
🧪 View experiment at: https://dagshub.com/Akarsh-Kumar-2004/reimagined-waffle.mlflow/#/experiments/0
